In [1]:
import pandas as pd
from tqdm import tqdm
import time

from bs4 import BeautifulSoup as bs


import requests

**Getting each season URL's**

In [ ]:
BASE_URL = 'https://breakingbad.fandom.com/wiki/' 

r = requests.get(BASE_URL + 'Category:Seasons_(Breaking_Bad)')

soup = bs(r.content, 'lxml')

In [2]:
all_items_S = soup.find_all('div', class_ = 'category-page__members-wrapper')[-1]
season_list = all_items_S.find_all('li', class_ = 'category-page__member')

season_link = []

for season in season_list:
    season_link.append((season.find('a').get('title'), BASE_URL + season.find('a').get('href')))

season_link

[('Season 1 (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki//wiki/Season_1_(Breaking_Bad)'),
 ('Season 2 (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki//wiki/Season_2_(Breaking_Bad)'),
 ('Season 3 (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki//wiki/Season_3_(Breaking_Bad)'),
 ('Season 4 (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki//wiki/Season_4_(Breaking_Bad)'),
 ('Season 5A (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki//wiki/Season_5A_(Breaking_Bad)'),
 ('Season 5B (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki//wiki/Season_5B_(Breaking_Bad)')]